# 地铁网络数据


In [1]:
import pandas as pd
import networkx as nx

import json

from pyecharts import options as opts
from pyecharts.charts import Graph

In [2]:
pd.set_option('future.no_silent_downcasting', True)

In [3]:
import sys
sys.path.append('../k_libs')
from subway_network import NetDataPivot, NetDegree, NetDataGraph, NetLength, CityNetworkAnalyzer

# 数据处理

In [4]:
df_row = pd.read_csv('../data/subway/ads_subway_cleaned.csv')
df_row

,city_id,city_name,city_pinyin,line_number_x,line_name,line_full_name,line_color,line_id,line_is_loop,line_is_show,...,target_st_longitude,target_st_latitude,st_cnt,uni_line_number,longitude_bd,latitude_bd,target_st_longitude_bd,target_st_latitude_bd,ll_distance,line_order
0,1100,北京,beijing,8,7号线,地铁7号线,F9BE58,110100023054,0,1,...,116.695495,39.856722,2,8,116.685663,39.854994,116.702126,39.862387,1.633,8
1,1100,北京,beijing,8,7号线,地铁7号线,F9BE58,110100023054,0,1,...,116.684340,39.863154,2,8,116.702126,39.862387,116.690939,39.868921,1.191,8
2,1100,北京,beijing,8,7号线,地铁7号线,F9BE58,110100023054,0,1,...,116.670792,39.863515,1,8,116.690939,39.868921,116.677315,39.869507,1.157,8
3,1100,北京,beijing,8,7号线,地铁7号线,F9BE58,110100023054,0,1,...,116.657379,39.863491,1,8,116.677315,39.869507,116.663823,39.869712,1.145,8
4,1100,北京,beijing,8,7号线,地铁7号线,F9BE58,110100023054,0,1,...,116.632160,39.863328,1,8,116.663823,39.869712,116.638568,39.869655,2.153,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7526,8200,澳门,aomen,3,氹仔线,氹仔线,008187,900000126430,0,1,...,113.552300,22.154190,1,3,113.563410,22.157302,113.558795,22.160146,0.565,103
7527,8200,澳门,aomen,2,石排湾线,石排湾线,786DAF,900000213458,0,1,...,NaN,NaN,1,2,113.573270,22.134229,NaN,NaN,NaN,102
7528,8200,澳门,aomen,2,石排湾线,石排湾线,786DAF,900000213458,0,1,...,113.566708,22.128488,2,2,113.574886,22.141501,113.573270,22.134229,0.828,102
7529,8200,澳门,aomen,1,横琴线,横琴线,E3002A,900000213462,0,1,...,NaN,NaN,1,1,113.556959,22.142970,NaN,NaN,NaN,101


# 特别说明

In [5]:
notes = (
    "本研究所用原始数据来源于网络，仅供参考。所有结构性指标均基于普通无向图模型计算，其中车站数量按照去重原则统计。鉴于部分城市的地铁网络存在孤立子网，相关结构性指标的计算均基于最大连通子图，以保证结果的合理性。"
)
city_notes = {
    "成都" : "成都地铁19号线建设区段为金星站至天府站，从天府站后起与成都地铁18号线共线运营至天府机场北站。成都地铁19号线表格中的车站数量不含共线段。",
}

# 全部城市网络特征


In [6]:
df_index = pd.read_csv('../data/subway/ads_subway_index.csv')
df_index


,序号,城市,线路数,车站数,区间数,平均度,最大度,换乘站数量,换乘站比例,密度,同配系数,平均聚类系数,是否连通图,直径,平均最短路径长度,全局效率,平均局部效率
0,1,北京,27,401,480,2.39,6,94,0.2344,0.00599,0.10849,0.00648,是,50,15.36,0.09472,0.00648
1,2,上海,21,413,496,2.40,8,88,0.2131,0.00583,0.19857,0.01356,是,44,15.05,0.09480,0.01384
2,3,广州,21,322,375,2.33,5,69,0.2143,0.00726,0.06376,0.00466,是,48,15.67,0.09588,0.00466
3,4,深圳,17,335,387,2.31,8,59,0.1761,0.00692,0.11148,0.00333,否,40,14.95,0.09933,0.00380
4,5,成都,15,336,389,2.32,6,64,0.1910,0.00695,-0.02319,0.00299,是,41,14.29,0.10079,0.00299
5,6,重庆,15,268,299,2.24,6,43,0.1610,0.00842,0.07740,0.00437,是,49,15.80,0.09782,0.00437
6,7,杭州,14,296,329,2.22,6,46,0.1554,0.00754,-0.04247,0.00225,是,64,18.39,0.08844,0.00225
7,8,南京,13,226,242,2.16,6,28,0.1250,0.00969,-0.03980,0.00000,是,49,16.25,0.09700,0.00000
8,9,郑州,13,235,271,2.31,6,47,0.2000,0.00986,-0.10257,0.00213,否,34,10.94,0.13319,0.00262
9,10,武汉,12,271,298,2.20,6,37,0.1365,0.00815,-0.00522,0.00332,是,46,15.99,0.09490,0.00332


# 可视化数据


## 城市数据

In [7]:
city_name = "深圳"

In [8]:
df_city = df_row[df_row['city_name'] == city_name]
df_city

,city_id,city_name,city_pinyin,line_number_x,line_name,line_full_name,line_color,line_id,line_is_loop,line_is_show,...,target_st_longitude,target_st_latitude,st_cnt,uni_line_number,longitude_bd,latitude_bd,target_st_longitude_bd,target_st_latitude_bd,ll_distance,line_order
5471,4403,深圳,shenzhen,7,6号线/光明线,地铁6号线(光明线),00AD8E,440300024046,0,1,...,113.867694,22.780928,1,7,113.854798,22.783886,113.874123,22.787198,2.015,7
5472,4403,深圳,shenzhen,7,6号线/光明线,地铁6号线(光明线),00AD8E,440300024046,0,1,...,113.880928,22.781610,1,7,113.874123,22.787198,113.887344,22.787948,1.359,7
5473,4403,深圳,shenzhen,7,6号线/光明线,地铁6号线(光明线),00AD8E,440300024046,0,1,...,113.907292,22.777684,1,7,113.897832,22.783707,113.913764,22.783729,1.629,7
5474,4403,深圳,shenzhen,7,6号线/光明线,地铁6号线(光明线),00AD8E,440300024046,0,1,...,113.991785,22.668097,1,7,113.956226,22.688955,113.998212,22.674406,4.639,7
5475,4403,深圳,shenzhen,7,6号线/光明线,地铁6号线(光明线),00AD8E,440300024046,0,1,...,114.016637,22.640394,1,7,114.011554,22.662653,114.023076,22.646611,2.129,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5874,4403,深圳,shenzhen,2,坪山云巴1号线,坪山云巴1号线,2249A3,900000191216,0,1,...,114.340490,22.708480,1,2,114.352001,22.716417,114.346974,22.714663,0.550,102
5875,4403,深圳,shenzhen,2,坪山云巴1号线,坪山云巴1号线,2249A3,900000191216,0,1,...,114.359490,22.687313,1,2,114.367519,22.688474,114.365929,22.693657,0.599,102
5876,4403,深圳,shenzhen,2,坪山云巴1号线,坪山云巴1号线,2249A3,900000191216,0,1,...,114.361191,22.709298,1,2,114.367979,22.709198,114.367643,22.715644,0.718,102
5877,4403,深圳,shenzhen,2,坪山云巴1号线,坪山云巴1号线,2249A3,900000191216,0,1,...,114.358206,22.691926,1,2,114.365929,22.693657,114.364650,22.698270,0.530,102


## 网络图
经纬度坐标图，信息可视化坐标图


## 线路间弦图

### 线路数据

In [9]:
def get_city_line_data(df_city):
    # 线路排序
    df_line = df_city[['city_pinyin', 'line_name', 'line_order', 'line_color']].drop_duplicates(subset='line_name').sort_values(by='line_order').reset_index(drop=True)
    # 将index复制为line_id列
    df_line['line_id'] = df_line.index + 1
    # 统计每个车站涉及的线路数
    station_line_counts = df_city.groupby('st_name')['line_name'].nunique()
    # 换乘站定义为涉及多条线路的车站
    transfer_stations = station_line_counts[station_line_counts > 1]
    # 计算df_city中，两两线路间的换乘站数量
    transfer_station_pairs = df_city[df_city['st_name'].isin(transfer_stations.index)]
    transfer_station_pairs = transfer_station_pairs.groupby('line_name')['st_name'].apply(list)
    transfer_station_pairs = transfer_station_pairs.reset_index()
    transfer_station_pairs['trans_st_count'] = transfer_station_pairs['st_name'].apply(len)
    # 将每条线路的换乘站数量添加到df_line中
    df_line = df_line.merge(transfer_station_pairs[['line_name', 'trans_st_count']], on='line_name', how='left')
    df_line['trans_st_count'] = df_line['trans_st_count'].fillna(0).astype(int)
    # 将trans_station_pairs中的st_name列添加到df_line中，列名为trans_stations
    df_line['trans_stations'] = df_line['line_name'].map(transfer_station_pairs.set_index('line_name')['st_name'])
    df_line['trans_stations'] = df_line['trans_stations'].fillna('[]')
    # line_color转为十六进制颜色代码
    df_line['line_color'] = "#" + df_line['line_color']

    return df_line

In [10]:
df_line = get_city_line_data(df_city)
df_line

,city_pinyin,line_name,line_order,line_color,line_id,trans_st_count,trans_stations
0,shenzhen,1号线/罗宝线,1,#00AB4F,1,13,"[桃园, 科学馆, 岗厦, 车公庙, 大剧院, 会展中心, 老街, 购物公园, 世界之窗, ..."
1,shenzhen,2号线/8号线,3,#B35A1F,2,13,"[景田, 岗厦北, 安托山, 黄贝岭, 海上世界, 后海, 市民中心, 大剧院, 华强北, ..."
2,shenzhen,3号线/龙岗线,4,#01A2E2,3,13,"[老街, 田贝, 红岭, 莲花村, 福田, 少年宫, 双龙, 华新, 石厦, 通新岭, 布吉..."
3,shenzhen,4号线/龙华线,5,#CC0000,4,8,"[上梅林, 会展中心, 深圳北站, 市民中心, 少年宫, 红山, 福民, 福田口岸]"
4,shenzhen,5号线/环中线,6,#96499A,5,11,"[太安, 宝安中心, 西丽, 灵芝, 前海湾, 布吉, 深圳北站, 五和, 赤湾, 前湾, ..."
5,shenzhen,6号线/光明线,7,#00AD8E,6,8,"[银湖, 松岗, 光明, 深圳北站, 通新岭, 八卦岭, 红山, 科学馆]"
6,shenzhen,6号线支线,8,#008187,7,1,[光明]
7,shenzhen,7号线/西丽线,9,#2249A3,8,13,"[西丽, 车公庙, 华新, 福民, 华强南, 田贝, 安托山, 华强北, 黄木岗, 红岭北,..."
8,shenzhen,9号线/梅林线,10,#86656D,9,11,"[前湾, 车公庙, 红岭, 粤海门, 上梅林, 红树湾南, 景田, 孖岭, 红岭北, 银湖,..."
9,shenzhen,10号线/坂田线,11,#EA6183,10,7,"[岗厦, 福民, 福田口岸, 岗厦北, 五和, 孖岭, 莲花村]"


### 线路间换乘站数量

In [11]:
def get_line_transfer_matrix(df_city):
    # 获取所有线路名
    lines = df_city.sort_values(by='line_order')['line_name'].unique()

    # 构建线路间换乘站数量的矩阵
    # 统计每个车站涉及的线路数
    station_line_counts = df_city.groupby('st_name')['line_name'].nunique()
    # 换乘站定义为涉及多条线路的车站
    transfer_stations = station_line_counts[station_line_counts > 1]
    transfer_station_set = set(transfer_stations.index)
    line_transfer = pd.DataFrame(0, index=lines, columns=lines)

    for line1 in lines:
        stations1 = set(df_city[df_city['line_name'] == line1]['st_name']) & transfer_station_set
        for line2 in lines:
            if line1 == line2:
                continue
            stations2 = set(df_city[df_city['line_name'] == line2]['st_name']) & transfer_station_set
            # 两条线路的换乘站交集数量
            num_common = len(stations1 & stations2)
            line_transfer.loc[line1, line2] = num_common
    # 新建index列,并将原index列移动到第一列,然后重命名为line_name
    line_transfer = line_transfer.reset_index().rename(columns={'index': 'line_name'})

    return line_transfer

In [12]:
line_transfer_matrix = get_line_transfer_matrix(df_city)
line_transfer_matrix

,line_name,1号线/罗宝线,2号线/8号线,3号线/龙岗线,4号线/龙华线,5号线/环中线,6号线/光明线,6号线支线,7号线/西丽线,9号线/梅林线,10号线/坂田线,11号线/机场线,12号线/南宝线,13号线/石岩线,14号线/东部快线,16号线/龙坪线,20号线,坪山云巴1号线
0,1号线/罗宝线,0,2,2,1,2,1,0,1,1,1,2,2,1,0,0,0,0
1,2号线/8号线,2,0,1,1,2,0,0,2,1,1,3,1,2,1,0,0,0
2,3号线/龙岗线,2,1,0,1,1,1,0,3,1,1,1,0,0,2,2,0,0
3,4号线/龙华线,1,1,1,0,1,2,0,1,1,2,0,0,0,0,0,0,0
4,5号线/环中线,2,2,1,1,0,1,0,2,1,1,1,1,0,1,0,0,0
5,6号线/光明线,1,0,1,2,1,0,1,1,1,0,1,1,0,0,0,0,0
6,6号线支线,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
7,7号线/西丽线,1,2,3,1,2,1,0,0,2,1,2,0,0,1,0,0,0
8,9号线/梅林线,1,1,1,1,1,1,0,2,0,1,2,1,1,0,0,0,0
9,10号线/坂田线,1,1,1,2,1,0,0,1,1,0,1,0,0,1,0,0,0


### 图数据

In [13]:
# 线路节点
line_nodes = []
for idx, row in df_line.iterrows():
    line_nodes.append({
        "id": str(row['line_id']),
        "name": row['line_name'],
        "symbolSize": row['trans_st_count']*2,
        'itemStyle': {
            "normal": {
                "color": '#' + row['line_color'],
                "opacity": 0.8
            }
        }
    })
line_nodes                    

[{'id': '1',
  'name': '1号线/罗宝线',
  'symbolSize': 26,
  'itemStyle': {'normal': {'color': '##00AB4F', 'opacity': 0.8}}},
 {'id': '2',
  'name': '2号线/8号线',
  'symbolSize': 26,
  'itemStyle': {'normal': {'color': '##B35A1F', 'opacity': 0.8}}},
 {'id': '3',
  'name': '3号线/龙岗线',
  'symbolSize': 26,
  'itemStyle': {'normal': {'color': '##01A2E2', 'opacity': 0.8}}},
 {'id': '4',
  'name': '4号线/龙华线',
  'symbolSize': 16,
  'itemStyle': {'normal': {'color': '##CC0000', 'opacity': 0.8}}},
 {'id': '5',
  'name': '5号线/环中线',
  'symbolSize': 22,
  'itemStyle': {'normal': {'color': '##96499A', 'opacity': 0.8}}},
 {'id': '6',
  'name': '6号线/光明线',
  'symbolSize': 16,
  'itemStyle': {'normal': {'color': '##00AD8E', 'opacity': 0.8}}},
 {'id': '7',
  'name': '6号线支线',
  'symbolSize': 2,
  'itemStyle': {'normal': {'color': '##008187', 'opacity': 0.8}}},
 {'id': '8',
  'name': '7号线/西丽线',
  'symbolSize': 26,
  'itemStyle': {'normal': {'color': '##2249A3', 'opacity': 0.8}}},
 {'id': '9',
  'name': '9号线/梅林线',
 

In [14]:
def get_line_links(df_city):
    line_transfer_matrix = get_line_transfer_matrix(df_city)
    # 线路连边，将line_transfer转换为连边格式，将线路名转换为df_line中的line_id,color使用df_line中的line_color
    line_links = []
    # 获取所有线路名
    lines = df_city.sort_values(by='line_order')['line_name'].unique()
    df_line = get_city_line_data(df_city)
    for i, row in line_transfer_matrix.iterrows():
        source_name = row['line_name']
        source_id = df_line.loc[df_line['line_name'] == source_name, 'line_id'].values[0]
        source_color = df_line.loc[df_line['line_name'] == source_name, 'line_color'].values[0]
        for target_name in lines[i:]:
            if source_name == target_name:
                continue
            value = row[target_name]
            if value > 0:
                target_id = df_line.loc[df_line['line_name'] == target_name, 'line_id'].values[0]
                line_links.append({
                    'source': str(source_id),
                    'target': str(target_id),
                    'source_name': source_name,
                    'target_name': target_name,
                    'value': int(value),
                    'lineStyle': {
                        'color': '#' + source_color,
                        'width': int(value),
                        'curveness': 0.2
                    }
                })
    return line_links

get_line_links(df_city)

[{'source': '1',
  'target': '2',
  'source_name': '1号线/罗宝线',
  'target_name': '2号线/8号线',
  'value': 2,
  'lineStyle': {'color': '##00AB4F', 'width': 2, 'curveness': 0.2}},
 {'source': '1',
  'target': '3',
  'source_name': '1号线/罗宝线',
  'target_name': '3号线/龙岗线',
  'value': 2,
  'lineStyle': {'color': '##00AB4F', 'width': 2, 'curveness': 0.2}},
 {'source': '1',
  'target': '4',
  'source_name': '1号线/罗宝线',
  'target_name': '4号线/龙华线',
  'value': 1,
  'lineStyle': {'color': '##00AB4F', 'width': 1, 'curveness': 0.2}},
 {'source': '1',
  'target': '5',
  'source_name': '1号线/罗宝线',
  'target_name': '5号线/环中线',
  'value': 2,
  'lineStyle': {'color': '##00AB4F', 'width': 2, 'curveness': 0.2}},
 {'source': '1',
  'target': '6',
  'source_name': '1号线/罗宝线',
  'target_name': '6号线/光明线',
  'value': 1,
  'lineStyle': {'color': '##00AB4F', 'width': 1, 'curveness': 0.2}},
 {'source': '1',
  'target': '8',
  'source_name': '1号线/罗宝线',
  'target_name': '7号线/西丽线',
  'value': 1,
  'lineStyle': {'color': '##00A

### 数据转为json

In [15]:
# 城市指标数据转为JSON
city_index = []
for i, row in df_index.iterrows():
    city_index.append({
        "city_name": row['城市'],
        "data": row.to_dict()
    })
with open(f'../data/subway/city_index.json', 'w', encoding='utf-8') as f:
    json.dump(city_index, f, ensure_ascii=False, indent=4)

In [16]:
# 线路数据转为json
city_list = df_index['城市'].unique().tolist()
city_line_data = []
for city_name in city_list:
    df_city = df_row[df_row['city_name'] == city_name]
    df_line = get_city_line_data(df_city)
    dict_line = df_line.to_dict(orient='records')
    city_dict = {
        'city_name': city_name,
        'lines_data': dict_line
    }
    city_line_data.append(city_dict)
with open(f'../data/subway/city_line_data.json', 'w', encoding='utf-8') as f:
    json.dump(city_line_data, f, ensure_ascii=False, indent=4)

In [17]:
# 线路间换乘关系连边数据
city_list = df_index['城市'].unique().tolist()
city_line_links = []
for city_name in city_list:
    df_city = df_row[df_row['city_name'] == city_name]
    line_count = df_city['line_name'].nunique()
    if line_count > 1:
        line_links = get_line_links(df_city)
        city_dict = {
            'city_name': city_name,
            'line_links': line_links
        }
        city_line_links.append(city_dict)
with open(f'../data/subway/city_line_links.json', 'w', encoding='utf-8') as f:
    json.dump(city_line_links, f, ensure_ascii=False, indent=4)

### 绘图

In [18]:
# 构建图表
net_graph = Graph(init_opts=opts.InitOpts(width="1000px", height="1000px")).add(
        "",
        nodes=line_nodes,
        links=line_links,
        # categories=categories,
        layout="circular",
        # layout="none",
        is_rotate_label=False,
        # linestyle_opts=opts.LineStyleOpts(color=report_color['line'], curve=0.1, opacity=0.4),
        label_opts=opts.LabelOpts(
            position="outside",  # 标签位置（会根据圆环自动调整）
            rotate=True,       # 标签旋转（沿圆环切线方向）
            color="auto", 
            # is_show=False,
            # distance=12,
            # font_size=14
            ),
        
    ).set_global_opts(
        title_opts=opts.TitleOpts(subtitle="* 为保证可视化效果，仅展示30部作品，及其主要演员。",  
                                  pos_bottom="20px", 
                                  pos_left="20px", 
                                #   subtitle_textstyle_opts=opts.TextStyleOpts(color=report_color['p'], font_size=16)),
    ))

# 获取图表 HTML 代码（不生成本地文件）
net_graph.render_notebook()

## 直径图
部分城市的地铁网络并非严格意义上的连通图。例如，长沙的磁悬浮机场线（S2 线）以及深圳的坪山云巴 1 号线均为独立线路，与城市主体地铁网并未形成连通。因此，在进行地铁网络建模与计算结构性指标时，通常需要基于最大连通子图进行分析，以避免孤立子图对平均最短路径、网络效率等指标造成干扰或失真。
本研究中将各类轨道交通线路统一统称为“地铁”。在若干城市的地铁系统中，整体网络并非严格意义上的单一连通图。例如，长沙的 S2 线（机场线）以及深圳的坪山云巴 1 号线均为独立运行线路，与城市主体地铁网络缺乏连接。若直接将这些孤立子网纳入整体网络建模，可能导致部分拓扑指标（如平均最短路径长度、全局效率等）产生偏差，从而影响对网络整体特性的准确刻画。因此，本研究在计算地铁网络的结构性指标时，统一基于最大连通子图进行分析，以确保所得结果能够更真实地反映城市地铁主体网络的结构特征与运行效率。
